In [1]:
from pathlib import Path
from indigo import Indigo
import numpy as np
import pandas as pd
import asyncio

from substrucure_finder import BucketsInitializer
from substrucure_finder import SearchEngine

import fp_utils
from fp_utils.finders import Finder
from fp_utils.tests import FinderSpeedTester
from fp_utils.catch_time import CatchTime

SyntaxError: 'await' outside async function (search_engine.py, line 31)

In [ ]:
fp_utils.settings.init_fp_utils()

In [ ]:
data_path = Path("../data/")

In [ ]:
zero_columns_name = '../data/zero_columns'

# Build structure

In [ ]:
data_name = ''

In [ ]:
initializer = BucketsInitializer(data_path / data_name, columns_count=100)

In [ ]:
(data_path / data_name / 'buckets').mkdir(exist_ok=True)

In [ ]:
%%time
initializer.init_buckets()

# Search Queries

In [ ]:
with open(zero_columns_name, 'r') as f:
    zero_columns = list(map(int, f.read().split()))
assert list(sorted(zero_columns)) == zero_columns

In [ ]:
full_columns = list(sorted(set(range(3736)) - set(zero_columns)))

In [ ]:
def bin_format(number, num_of_bits=3736):
    fp = list(map(int, list(bin(int(number, 16))[2:].zfill(num_of_bits))))
    return np.fromiter(fp, dtype=int)

In [ ]:
def smiles_to_fingerprint(smiles):
    indigo_mol_to_test = Indigo().loadMolecule(smiles)
    full_fp = bin_format(indigo_mol_to_test.fingerprint("sub").toString())
    return np.fromiter(pd.Series(full_fp)[full_columns].values, dtype=bool)

In [ ]:
@CatchTime("search")
def search(smiles: str, finder: Finder, ans_count=None):
    fp = smiles_to_fingerprint(smiles)
    return list(finder.find(fp, ans_count))

In [ ]:
class ComplexFinder(SearchEngine, Finder):   
    def find_all(self, fp):
        return self.search(fp)

In [ ]:
class AsyncFinder(SearchEngine, Finder):   
    def find_all(self, fp):
        return self.async_search(fp)

In [ ]:
# mcc100_2kk = ComplexFinder(data_path / '2kk_100_mcc')
# mcc100_2kk_async = AsyncFinder(data_path / '2kk_100_mcc')
# mcc100_17kk_heur2000 = ComplexFinder(data_path / '17kk_100_heur2000')
# mcc100_17kk_heur2000_async = AsyncFinder(data_path / '17kk_100_heur2000')
# mcc100_2kk_heur1000 = ComplexFinder(data_path / '2kk_100_heur1000')
# mcc100_2kk_heur1500 = ComplexFinder(data_path / '2kk_100_heur1500')
# mcc100_2kk_heur1700 = ComplexFinder(data_path / '2kk_100_heur1700')

In [ ]:
# mcc100_17kk = ComplexFinder(data_path / '17kk_100_heur2000')

In [ ]:
with open('../data/pubchem_994_queries.txt', 'r') as f:
    queries = list(map(lambda x: x.split()[0], f.read().strip().split('\n')))

In [ ]:
len(queries), queries[:10]

In [ ]:
queries_fp = [smiles_to_fingerprint(smiles) for smiles in queries]

In [ ]:
import random

In [ ]:
queries_30 = random.sample(queries_fp, 30)

In [ ]:
mcc100_17kk_heur2000 = ComplexFinder(data_path / '17kk_100_heur2000')

In [ ]:
# %%time 
# stat_sync = FinderSpeedTester([mcc100_17kk_heur2000]).test_all(queries_30, verbose=True, ans_count=10)

In [ ]:
mcc100_17kk_heur2000_async = AsyncFinder(data_path / '17kk_100_heur2000')

In [ ]:
%%time 
stat_async = FinderSpeedTester([mcc100_17kk_heur2000_async]).test_all(queries_30, verbose=True, ans_count=10)

In [ ]:
stat = stat_sync + stat_async

In [ ]:
stat

In [ ]:
stat.as_boxplot()

In [ ]:
stat.as_plot()

In [ ]:
from fp_utils.draw import MoleculeDrawer

In [ ]:
query = queries[0]
MoleculeDrawer.draw_one(query)

In [ ]:
answers = search(query, mcc100_17kk_heur2000_async, ans_count=10)

In [ ]:
len(answers)

In [ ]:
MoleculeDrawer.draw_many(answers, figsize=(20, 20), shape=(2, 3))

In [ ]:
answers = search(query, mcc100_17kk_heur2000, ans_count=10)
MoleculeDrawer.draw_many(answers, figsize=(20, 20), shape=(2, 3))